<a href="https://colab.research.google.com/github/mavy731/UIUC-MSFE-FALL20-FIN553/blob/main/Assignment1_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 100000  # Number of simualted paths
m = 50      # number of exercise dates
T = 1       # maturity
order = 6   # Polynmial order
Δt = T / m  # interval between two exercise dates

In [25]:
def one_hot(array, depth):
    return tf.eye(depth)[array]

#Scale x
def scale(x):
    x_scale=(x-tf.math.reduce_min(x,axis=0))/(tf.math.reduce_max(x,axis=0)-tf.math.reduce_min(x,axis=0))
    return x_scale

# recursive formulation
def chebyshev_basis(x, k):
    B=tf.stack([tf.ones([n],dtype=tf.float32),x],1)
    for i in range(2, k):
        Bn=2*S0*B[::,i-1]-B[::,i-2]
        Bn=tf.expand_dims(Bn,1)
        B=tf.concat([B,Bn],1)
    return B

# simulates the stock price evolution
def advance(S, r, σ, Δt, n):
    dB = tf.sqrt(Δt) * tf.random.normal([n])
    out = S + r * S * Δt + σ * S * dB
    return out

In [26]:
tf.random.set_seed(0)
S0 = Spot * tf.ones([n],dtype=tf.float32)
S = {0: S0}

for t in range(m):
   S[t + 1] = advance(S[t], r, σ, Δt, n)

discount = tf.exp(-r * Δt)
CFL = {t: tf.maximum(0., K - S[t]) for t in S.keys()}
value_tp1 = CFL[m] * discount
CV = {m: tf.ones([n],dtype=tf.float32)}

In [22]:
X=chebyshev_basis(scale(S[49]), order)
Y= tf.expand_dims(value_tp1,1)
rhs=tf.transpose(X) @ Y
matrix=tf.transpose(X) @ X
Θ =tf.linalg.solve(matrix, rhs)
CV[49]=tf.reshape(X @ Θ,[-1,])
exercise = CFL[49] > CV[49]
value_t = tf.where(exercise, CFL[49], value_tp1)
print(value_t)

tf.Tensor([6.2191825 0.        8.81735   ... 5.3614364 0.        0.       ], shape=(100000,), dtype=float32)


In [28]:
for t in range(m - 1, 0, -1):
  X=chebyshev_basis(scale(S[t]), order)
  Y= tf.expand_dims(value_tp1,1)
  rhs=tf.transpose(X) @ Y
  matrix=tf.transpose(X) @ X
  Θ =tf.linalg.solve(matrix, rhs)
  CV[t] = tf.reshape(X @ Θ,[-1,])
  exercise = CFL[t] > CV[t]
  value_t = tf.where(exercise, CFL[t], value_tp1)
  value_tp1 = discount * value_t

In [30]:
POF = {t: tf.where(CV[t] < CFL[t], CFL[t], 0) for t in range(1, m + 1)}

In [5]:
POF = {t: tf.where(CV[t] < CFL[t], CFL[t], 0) for t in range(1, m + 1)}
POF = np.vstack(POF.values())
idx_payoffs = tf.argmax(POF > 0, axis=0)
FPOF = one_hot(idx_payoffs, m).T * POF
m_range = tf.array(range(0, m)).reshape(-1, 1)
dFPOF = FPOF * tf.exp(-r * m_range * Δt)
PRICE = dFPOF.sum() / n

In [ ]:
# LSMC algorithm
def compute_price(order, Spot, σ, K, r):
    tf.random.set_seed(0)
    S0 = Spot * tf.ones([n],dtype=tf.float32)
    S = {0: S0}

    for t in range(m):
        S[t + 1] = advance(S[t], r, σ, Δt, n)

    discount = tf.exp(-r * Δt)
    CFL = {t: tf.maximum(0., K - S[t]) for t in S.keys()}
    value_tp1 = CFL[m] * discount
    CV = {m: tf.zeros_like(S0)}

    for t in range(m - 1, 0, -1):
        X = chebyshev_basis(scale(S[t]), order)
        Y = value_tp1
        # regression to estimate the continuation value
        Θ = tf.linalg.solve(X.T @ X, X.T @ Y)
        CV[t] = X @ Θ
        exercise = CFL[t] > CV[t]
        value_t = tf.where(exercise, CFL[t], value_tp1)
        value_tp1 = discount * value_t

  
    POF = {t: tf.where(CV[t] < CFL[t], CFL[t], 0) for t in range(1, m + 1)}
    POF = np.vstack(POF.values())
    idx_payoffs = tf.argmax(POF > 0, axis=0)
    FPOF = one_hot(idx_payoffs, m).T * POF
    m_range = tf.array(range(0, m)).reshape(-1, 1)
    dFPOF = FPOF * tf.exp(-r * m_range * Δt)
    PRICE = dFPOF.sum() / n
    return PRICE

print(compute_price(order, Spot, σ, K, r))
test = compute_price(order, Spot, σ, K, r)